In [1]:
import os
import time

In [2]:
#Size of the file
os.path.getsize(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv")

2859504349

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df2 = dd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv")
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.022001028060913086 sec


In [4]:
import pandas as pd
start = time.time()
df2 = pd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv")
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  32.354231119155884 sec


In [5]:
import modin.pandas as pd
import ray
import time
ray.shutdown()
ray.init()
start = time.time()
df2 = pd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv")
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2023-02-11 16:01:17,661	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Read csv with modin and ray:  34.69835424423218 sec


In [6]:
from dask import dataframe as dd
import pandas as pd
df2 = pd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv", sep=',', dtype={'a': object})
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


In [7]:
#No. of Rows
len(df2.index)

3000000

In [8]:
#No, of Columns
len(df2.columns)

10

In [9]:
# remove special characters
df2.columns=df2.columns.str.replace('[#,@,&]','')

In [10]:
#To remove white space from columns
df2.columns = df2.columns.str.replace(' ', '')

In [11]:
data=df2.columns
data

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [12]:
%%writefile utility.py
#VALIDATING
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import configparser

def read_config_file(filepath='store.yaml'):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df2,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df2.columns = df2.columns.str.lower()
    df2.columns = df2.columns.str.replace('[^\w]','_',regex=True)
    df2.columns = list(map(lambda x: x.strip('_'), list(df2.columns)))
    df2.columns = list(map(lambda x: replacer(x,'_'), list(df2.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df2.columns =list(map(lambda x: x.lower(), list(df2.columns)))
    df2 = df2.reindex(sorted(df2.columns), axis=1)
    if len(df2.columns) == len(expected_col) and list(expected_col)  == list(df2.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df2.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df2.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df2 columns: {df2.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utility.py


In [13]:
%%writefile store.yaml

config_data = file_type: csv
dataset_name: Books_rating
file_name: Books_rating
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    -Id 
    -Title 
    -Price
    -User_id
    -profileName 
    -review/helpfulness 
    -review/score 
    -review/time
    -review/summary
    -review/text


Writing store.yaml


In [14]:
# Reading config file
import utility as util
import yaml
config_data = util.read_config_file("store.yaml")

In [15]:
#data of config file
config_data

{'config_data = file_type': 'csv',
 'dataset_name': 'Books_rating',
 'file_name': 'Books_rating',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': '-Id -Title -Price -User_id -profileName -review/helpfulness -review/score -review/time -review/summary -review/text'}

In [16]:
# Reading process of the file using Dask
from dask import dataframe as dd
import pandas as pd

df2_sample = pd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv", sep=',' , dtype={'Id': object, 'Title': object,'Price':float, 'User_id':object, 'profile_name':object, 'review/helpfulness':object, 'review/score':float, 'review/time':int, 'review/summary': object, 'review/text':object}, encoding="UTF-8")

df2_sample.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [17]:
#file_type = config_data['file_type']
source_file = r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv"
#+ config_data['file_name'] + f'.{file_type}'

In [18]:
import pandas as pd
df2 = pd.read_csv(source_file,config_data['inbound_delimiter'])
df2.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [19]:
#validating the header of the file
util.col_header_val(df2,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['id', 'review_time', 'review_helpfulness', 'profilename', 'review_summary', 'price', 'user_id', 'review_score', 'title', 'review_text']
Following YAML columns are not in the file uploaded ['l', 'a', 'c', 'h', 'w', 'e', 'd', ' ', '-', 'r', 'm', 'f', 'x', 'n', '/', 'i', 'p', '_', 'o', 'y', 's', 'v', 't', 'u']


0

In [20]:
print("columns of files are:" ,df2.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'title', 'price', 'user_id', 'profilename', 'review_helpfulness',
       'review_score', 'review_time', 'review_summary', 'review_text'],
      dtype='object')
columns of YAML are: -Id -Title -Price -User_id -profileName -review/helpfulness -review/score -review/time -review/summary -review/text


In [21]:
if util.col_header_val(df2,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['id', 'review_time', 'review_helpfulness', 'profilename', 'review_summary', 'price', 'user_id', 'review_score', 'title', 'review_text']
Following YAML columns are not in the file uploaded ['l', 'a', 'c', 'h', 'w', 'e', 'd', ' ', '-', 'r', 'm', 'f', 'x', 'n', '/', 'i', 'p', '_', 'o', 'y', 's', 'v', 't', 'u']
validation failed


In [22]:
import datetime
import csv
import gzip

from dask import dataframe as dd
import pandas

df2 = pd.read_csv(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv",sep=',',header=0,dtype={'a': object},encoding="UTF-8")
df2.head()
# Write csv in gz format in pipe separated text file (|)
df2.to_csv('Books_rating.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          lineterminator='\n')

In [23]:
import os
path = r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data"
entries = os.listdir(path)
count = 0
for entry in entries:
    if entry.endswith(".gz"):
        count += 1

print("Number of .gz files in the directory:", count)

Number of .gz files in the directory: 1


In [24]:
#No. of Rows
print("Total number of rows:", len(df2.index))

Total number of rows: 3000000


In [25]:
#No, of Columns
print("Total number of columns:", len(df2.columns))

Total number of columns: 10


In [26]:
#size of the gz format folder
print("File size is:", os.path.getsize(r"C:\Users\shrut\Desktop\Data Glacier Stuff\Data\Books_rating.csv.gz"))

File size is: 1060588862


(_deploy_ray_func pid=13340) 2023-02-11 16:11:14,474	ERROR import_thread.py:75 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
(_deploy_ray_func pid=13340) 	status = StatusCode.UNKNOWN
(_deploy_ray_func pid=13340) 	details = "Stream removed"
(_deploy_ray_func pid=13340) 	debug_error_string = "{"created":"@1676149874.444000000","description":"Error received from peer ipv4:127.0.0.1:60905","file":"src/core/lib/surface/call.cc","file_line":1064,"grpc_message":"Stream removed","grpc_status":2}"
(_deploy_ray_func pid=13340) >
(_deploy_ray_func pid=17464) 2023-02-11 16:11:14,458	ERROR import_thread.py:75 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
(_deploy_ray_func pid=17464) 	status = StatusCode.UNKNOWN
(_deploy_ray_func pid=17464) 	details = "Stream removed"
(_deploy_ray_func pid=17464) 	debug_error_string = "{"created":"@1676149874.444000000","description":"Error received from peer ipv4:127.0.0.1:60905","file":"src/core/lib/surface/call.